In [1]:
import datetime
from dateutil.parser import parse as parse_datetime
import re
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import smtplib
from email.mime.text import MIMEText
from pytz import UTC, timezone
import spacy
import nltk
from nltk.tokenize import word_tokenize

In [2]:
nlp = spacy.load("en_core_web_sm")

scope = ['https://www.googleapis.com/auth/calendar']
rec_mail = "jasmakhija1234@gmail.com"
conf_mail = "religious7903@gmail.com"
conf_app_pwd = "12 digit code"  # Replace with actual App Password
mail_reg = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
ist = timezone('Asia/Kolkata')
st = 9
end_st = 16

In [3]:
def auth_google_calendar():
    creds = None
    try:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', scope)
        creds = flow.run_local_server(port=3000)
        print("Successfully authenticated with Google Calendar")
    except FileNotFoundError:
        print("Error: 'credentials.json' file not found.")
        return None
    except Exception as e:
        print(f"Error during authentication: {e}")
        return None
    return build('calendar', 'v3', credentials=creds)

In [4]:
# import os.path

# creds = None
# if os.path.exists('token.json'):
#     creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# if not creds or not creds.valid:
#     flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
#     creds = flow.run_local_server(port=3000)
#     with open('token.json', 'w') as token_file:
#         token_file.write(creds.to_json())

In [5]:
def check_slot_availability(service, start_time, end_time):
    if service is None:
        return False
    events_result = service.events().list(
        calendarId='primary', timeMin=start_time.isoformat(), timeMax=end_time.isoformat(),
        singleEvents=True, orderBy='startTime'
    ).execute()
    return len(events_result.get('items', [])) == 0


In [6]:
def get_recruiter_free_slots(service, start_datetime, end_datetime):
    if service is None:
        return []
    events_result = service.events().list(
        calendarId='primary', timeMin=start_datetime.isoformat(), timeMax=end_datetime.isoformat(),
        singleEvents=True, orderBy='startTime'
    ).execute()
    events = events_result.get('items', [])
    free_slots = []
    current_time = start_datetime


    
    if not events:
        return [(start_datetime, end_datetime)]



    
    first_event_start = datetime.datetime.fromisoformat(events[0]['start']['dateTime'].replace('Z', '')).replace(tzinfo=UTC)
    if current_time < first_event_start:
        free_slots.append((current_time, first_event_start))


    
    for i in range(len(events)-1):
        current_end = datetime.datetime.fromisoformat(events[i]['end']['dateTime'].replace('Z', '')).replace(tzinfo=UTC)
        next_start = datetime.datetime.fromisoformat(events[i+1]['start']['dateTime'].replace('Z', '')).replace(tzinfo=UTC)
        if current_end < next_start:
            free_slots.append((current_end, next_start))

    
    last_event_end = datetime.datetime.fromisoformat(events[-1]['end']['dateTime'].replace('Z', '')).replace(tzinfo=UTC)
    if last_event_end < end_datetime:
        free_slots.append((last_event_end, end_datetime))


    
    filtered_slots = []
    for start, end in free_slots:
        local_start = start.astimezone(ist)
        local_end = end.astimezone(ist)
        if local_start.hour >= end_st or local_end.hour <= st:
            continue
        adjusted_start = max(start, ist.localize(datetime.datetime.combine(local_start.date(), datetime.time(st, 0))).astimezone(UTC))
        adjusted_end = min(end, ist.localize(datetime.datetime.combine(local_end.date(), datetime.time(end_st, 0))).astimezone(UTC))
        if adjusted_start < adjusted_end:
            filtered_slots.append((adjusted_start, adjusted_end))
    
    return filtered_slots

In [7]:
def parse_availability(text):
    text = text.lower().strip()
    today = datetime.datetime.now(ist).date()
    now_utc = datetime.datetime.now(UTC)
    tokens = word_tokenize(text)
    doc = nlp(text)

    target_date = None
    target_time = None
    is_next = "next" in tokens

    weekdays = {
        'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3, 'friday': 4, 'saturday': 5, 'sunday': 6,
        'mon': 0, 'tue': 1, 'wed': 2, 'thu': 3, 'fri': 4, 'sat': 5, 'sun': 6
    }

    for ent in doc.ents:
        if ent.label_ == "DATE":
            try:
                dt = parse_datetime(ent.text, fuzzy=True, default=datetime.datetime.combine(today, datetime.time(0, 0)))
                parsed_date = dt.date()
                if parsed_date < today and not is_next:
                    parsed_date = parsed_date.replace(year=today.year + 1) if "next" not in text else parsed_date + datetime.timedelta(days=7)
                target_date = parsed_date
            except ValueError:
                continue
        elif ent.label_ == "TIME":
            try:
                dt = parse_datetime(ent.text, fuzzy=True, default=datetime.datetime.combine(today, datetime.time(0, 0)))
                target_time = dt.time()
            except ValueError:
                continue

    # Step 2: POS and dependency parsing
    for i, token in enumerate(doc):
        if token.text in weekdays and token.pos_ in ["NOUN", "PROPN"]:
            days_ahead = weekdays[token.text] - today.weekday()
            if days_ahead <= 0:
                days_ahead = days_ahead + 7
            if is_next or (i > 0 and doc[i-1].text == "next"):
                days_ahead += 7
            target_date = today + datetime.timedelta(days=days_ahead)

        if token.text == "today" and token.pos_ == "NOUN":
            target_date = today
        elif token.text == "tomorrow" and token.pos_ == "NOUN":
            target_date = today + datetime.timedelta(days=1)

        if token.pos_ == "NUM" and token.head.text in ["am", "pm"]:
            try:
                hour = int(token.text)
                if token.head.text == "pm" and hour < 12:
                    hour = hour + 12
                elif token.head.text == "am" and hour == 12:
                    hour = 0
                target_time = datetime.time(hour, 0)
            except ValueError:
                continue

        if token.text in ["morning", "afternoon"] and token.pos_ == "NOUN":
            time_map = {'morning': 10, 'afternoon': 14}
            target_time = datetime.time(time_map[token.text], 0)

    if not target_time:
        time_match = re.search(r'(\d{1,2})(?::(\d{2}))?\s*(am|pm)?', text)
        if time_match:
            hour = int(time_match.group(1))
            period = time_match.group(3)
            if period == "pm" and hour < 12:
                hour = hour + 12
            elif period == "am" and hour == 12:
                hour = 0
            if 0 <= hour < 24:
                target_time = datetime.time(hour, 0)

    if not target_date:
        target_date = today
    if not target_time:
        target_time = datetime.time(st, 0)

    local_dt = ist.localize(datetime.datetime.combine(target_date, target_time))
    utc_dt = local_dt.astimezone(UTC)

    # print(f"Debug - Parsed: {local_dt.strftime('%Y-%m-%d %I:%M %p %Z')} -> {utc_dt.strftime('%Y-%m-%d %I:%M %p UTC')}")

    if (local_dt.hour < st or local_dt.hour > end_st or
        utc_dt <= now_utc):
        return None

    return utc_dt

In [8]:
def get_valid_availability():
    while True:
        availability = input("When are you available? (e.g., 'Monday 2pm', '15 March 2025 14:00', 'tomorrow 3pm', between 9 AM and 4 PM): ")
        target_datetime = parse_availability(availability)
        
        if target_datetime:
            end_time = target_datetime + datetime.timedelta(hours=1)
            if end_time.astimezone(ist).hour > end_st:
                print("Meeting must end by 4 PM IST. Please choose an earlier time.")
                continue
            return target_datetime
        else:
            print("Invalid input, time outside 9 AM to 4 PM IST, or in the past. Please try again.")


In [9]:
def create_calendar_event(service, start_time, end_time, candidate_email):
    if service is None:
        return None
    event = {
        'summary': 'Interview with Candidate',
        'start': {'dateTime': start_time.isoformat(), 'timeZone': 'UTC'},
        'end': {'dateTime': end_time.isoformat(), 'timeZone': 'UTC'},
        'attendees': [{'email': rec_mail}, {'email': candidate_email}],
        'reminders': {'useDefault': False, 'overrides': [{'method': 'email', 'minutes': 30}, {'method': 'popup', 'minutes': 10}]},
        'conferenceData': {'createRequest': {'requestId': f"{start_time.strftime('%Y%m%d%H%M%S')}", 'conferenceSolutionKey': {'type': 'hangoutsMeet'}}}
    }
    
    try:
        return service.events().insert(calendarId='primary', body=event, conferenceDataVersion=1, sendUpdates='all').execute()
    except HttpError as error:
        print(f"An error occurred while creating the event: {error}")
        return None

In [10]:
def send_email(candidate_email, start_time, end_time, meet_link):
    local_start = start_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')
    local_end = end_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')
    
    msg_body = f"Interview scheduled from {local_start} to {local_end}\nJoin the meeting here: {meet_link}"
    msg = MIMEText(msg_body)
    msg['Subject'] = 'Interview Scheduled - Google Meet Link'
    msg['From'] = conf_mail
    msg['To'] = candidate_email
    
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(conf_mail, conf_app_pwd)
            server.send_message(msg)
        return True
    except smtplib.SMTPException as e:
        print(f"SMTP Error: {e}")
        return False

In [11]:
def main():
    service = auth_google_calendar()
    if service is None:
        print("Failed to authenticate with Google Calendar. Exiting.")
        return
    
    while True:
        print("\nAI Scheduling Bot")
        print("Enter time in your local timezone (Asia/Kolkata)")
        print("Examples: 'Monday 2pm', '15 March 2025 14:30', 'tomorrow 3pm' (between 9 AM and 4 PM)")
        
        candidate_email = input("Enter your email: ").strip()
        if not re.match(mail_reg, candidate_email):
            print("Invalid email format. Please enter a valid email address.")
            continue
        
        target_datetime = get_valid_availability()
        if not target_datetime:
            continue
        
        meeting_duration = datetime.timedelta(hours=1)
        requested_end = target_datetime + meeting_duration
        
        day_start = ist.localize(datetime.datetime.combine(target_datetime.astimezone(ist).date(), datetime.time(st, 0))).astimezone(UTC)
        day_end = ist.localize(datetime.datetime.combine(target_datetime.astimezone(ist).date(), datetime.time(end_st, 0))).astimezone(UTC)
        
        if check_slot_availability(service, target_datetime, requested_end):
            available_slot = (target_datetime, requested_end)
        else:
            free_slots = get_recruiter_free_slots(service, day_start, day_end)
            available_slot = None
            
            for slot_start, slot_end in free_slots:
                slot_duration = slot_end - slot_start
                if slot_start <= target_datetime and slot_end >= requested_end and slot_duration >= meeting_duration:
                    available_slot = (target_datetime, requested_end)
                    break
            
            if not available_slot:
                print(f"Sorry, the requested time "
                      f"{target_datetime.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')} is not available.")
                print("Here are the available slots:")
                for start, end in free_slots:
                    print(f"- {start.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')} to "
                          f"{end.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')}")
                continue
        
        start_time, end_time = available_slot
        
        if not check_slot_availability(service, start_time, end_time):
            print("Slot  unavailable  Please try again.")
            continue
        
        event = create_calendar_event(service, start_time, end_time, candidate_email)
        
        if event and 'id' in event:
            meet_link = event.get('hangoutLink', 'No Google Meet link generated')
            print(f"Meeting scheduled successfully: "
                  f"{start_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')} to "
                  f"{end_time.astimezone(ist).strftime('%Y-%m-%d %I:%M %p %Z')}")
            print(f"Google Meet link: {meet_link}")
            
            email_sent = send_email(candidate_email, start_time, end_time, meet_link)
            if email_sent:
                print("Confirmation email sent with Google Meet link!")
            else:
                print("Failed to send confirmation email")
        else:
            print("Failed to schedule the meeting in Google Calendar")
        
        if input("Schedule another meeting? (y/n): ").lower() != 'y':
            break

In [12]:
main()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=998403459185-kma8aj7rp8skb481irbr751smpnctv7b.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=RgGKg3wIqwx6NA5uKuRBjNT86Q6RHC&access_type=offline
Successfully authenticated with Google Calendar

AI Scheduling Bot
Enter time in your local timezone (Asia/Kolkata)
Examples: 'Monday 2pm', '15 March 2025 14:30', 'tomorrow 3pm' (between 9 AM and 4 PM)


Enter your email:  jasmakhija123456@gmail.com
When are you available? (e.g., 'Monday 2pm', '15 March 2025 14:00', 'tomorrow 3pm', between 9 AM and 4 PM):  next wed 12pm


Slot  unavailable  Please try again.

AI Scheduling Bot
Enter time in your local timezone (Asia/Kolkata)
Examples: 'Monday 2pm', '15 March 2025 14:30', 'tomorrow 3pm' (between 9 AM and 4 PM)


Enter your email:  jasmakhija123456@gmail.com
When are you available? (e.g., 'Monday 2pm', '15 March 2025 14:00', 'tomorrow 3pm', between 9 AM and 4 PM):  next thu 2pm


Meeting scheduled successfully: 2025-03-27 02:00 PM IST to 2025-03-27 03:00 PM IST
Google Meet link: https://meet.google.com/xir-fdgp-nzu
SMTP Error: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-225c6bd5d97sm62554355ad.259 - gsmtp')
Failed to send confirmation email


Schedule another meeting? (y/n):  n
